In [12]:
%matplotlib qt
%load_ext autoreload
%autoreload 2
from local_plot import *
from utils import *


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
import numpy as np
from scipy.interpolate import interp1d

class Trajectory:
    def __init__(self, t, pos, quat):
        self.t = t
        self.pos = pos
        self.quat = quat
        self.ypr = numpy.apply_along_axis(quat2eulers_arr, 1, quat)
        self.interp()

    def interp(self):
        self.pos_func = interp1d(self.t, self.pos, axis=0,bounds_error=False,fill_value="extrapolate")
        self.ypr_func = interp1d(self.t, self.ypr, axis=0,bounds_error=False,fill_value="extrapolate")
    
    def length(self):
        dp = np.diff(self.pos, axis=0)
        length = np.sum(np.linalg.norm(dp,axis=1))
        return length
    
    def resample_ypr(self, t):
        return self.ypr_func(t)
    
    def resample_pos(self, t):
        return self.pos_func(t)

def read_path_from_csv(path, t0=None):
    arr = np.loadtxt(path)
    t = arr[:, 0]
    if t0 is None:
        t0 = t[0]
    t = t - t0
    pos = arr[:, 1:4]
    quat = arr[:, 4:]
    return Trajectory(t, pos, quat), t0

def read_paths(folder, nodes, prefix="d2vins", t0=None):
    ret = {}
    for drone_id in nodes:
        ret[drone_id], t0 = read_path_from_csv(f"{folder}/{prefix}_{drone_id}.csv", t0)
    return ret, t0


In [14]:
nodes = {1, 2}
output_folder = "/home/xuhao/Dropbox/data/d2slam/tum_datasets/outputs/d2vins/swarm1"
data_folder="/home/xuhao/Dropbox/data/d2slam/tum_datasets/"
paths_gt, t0 = read_paths(data_folder, nodes, prefix="groundtruth")
paths, _ = read_paths(output_folder, nodes, t0=t0)
plot_fused(nodes, paths, poses_gt=paths_gt)
display(plot_fused_err(nodes, paths, paths_gt,dte=140, show=True, outlier_thres_yaw=0.1))
plot_relative_pose_err(1, [2], paths, paths_gt,dte=140, show=False, outlier_thres=0.2, outlier_yaw_thres=0.1)

Drone,ATE Pos,ATE Att,Cov/m: x,y,z,PGO:ATE Pos,ATE Att
1by1,0.124,1.616,1.2e-04,1.0e-04,1.3e-04,nan,nan°
2by1,0.129,2.266,9.5e-05,1.1e-04,6.2e-05,nan,nan°


Relative,EST RMSE: Pos,Ang,BIAS: Pos,Ang
1->2,"0.033,0.032,0.021",1.20°,"0.002,+0.004,+0.000",+2.32°
